In [13]:
from __future__ import print_function
import sys, os
sys.path.insert(0, os.getcwd()+'/../deepx/')

import pandas as pd
import matplotlib.pyplot as plt
from time import time
from IPython.display import display, clear_output
#%matplotlib inline

import numpy as np
from sklearn.datasets import fetch_mldata

from deepx.helper.helper import load_mnist, load_cifar10, load_cifar100, plot_results

In [ ]:
results = pd.read_csv(os.getcwd()+'/../deepx/examples/results 10 l:3, e:70')

fig, ax1 = plt.subplots()
ax2 = ax1.twiny()
ax3 = ax1.twinx()
    
plot_results(results, plt, ax1, ax2, ax3)
plt.show(block = True)
results

In [ ]:
from __future__ import print_function
import sys, os
sys.path.insert(0, os.getcwd()+'/..')

import pandas as pd
import matplotlib.pyplot as plt
from time import time
from IPython.display import display, clear_output
#%matplotlib inline

import numpy as np
from sklearn.datasets import fetch_mldata

from deepx.helper.helper import load_mnist, load_cifar, load_cifar10, plot_results
from deepx.nn import *
from deepx.loss import *
from deepx.optimize import *

#np.random.seed(0)

import seaborn as sns
sns.set_style("darkgrid")
fig, ax1 = plt.subplots()
ax2 = ax1.twiny()
ax3 = ax1.twinx()

def pipeline(Xtrain, ytrain, Xtest, ytest, rms_tanlu, rms_opt, epochs):
    def train(n_iter, lr=1):
        alphas = [a for a in rms_tanlu.parameters if str(a) == 'alpha']
        results = pd.DataFrame(columns = ['Iteration', 'Time', 'Loss Tanlu', 'Loss Optimal', \
            'Error Tanlu', 'Error Optimal'] + ['alpha_'+str(j+1) for j in range(len(alphas))])

        def compare_archs(rms):
            losses = []
            errors = []
            def test_alphas(net):
                loss_buf = net.train(Xtrain[u:u+minibatch], ytrain[u:u+minibatch], lr)
                error_buf = test(net.model, stride = 10)
                return loss_buf, error_buf

            loss, err = test_alphas(rms)
            loss_opt, err_opt = test_alphas(rms_opt)

            r = (i, round((time() - t0)/60.0,1))
            r += tuple([round(x,4) for x in [loss.max(), loss_opt.max(), err, err_opt]])
            r += tuple([round(a.get_value().item(),2) for a in rms.parameters if str(a) == 'alpha'])
            results.loc[results.shape[0]] = r
            print(results.tail(1).to_string(index=False))
            plot_results(results, plt, ax1, ax2, ax3)


        t0 = time()
        minibatch = 128
        freq = 1000

        for i in range(n_iter):
            u = np.random.choice(np.arange(Xtrain.shape[0]))
            rms_tanlu.train(Xtrain[u:u+minibatch], ytrain[u:u+minibatch], lr)
            rms_opt.train(Xtrain[u:u+minibatch], ytrain[u:u+minibatch], lr)
            if (i) % freq == 0:
                compare_archs(rms_tanlu)
        fig.savefig("../deepx/helper/Results/tanlu vs tanh+relu "+str(n_iter), dpi=100)
        return results

    def test(model, stride = 1):
        preds = model.predict(Xtest[::stride]).argmax(axis=1)
        return 1 - np.mean(preds == ytest[::stride].argmax(axis=1))

    n_iter = 1000*epochs
    results = train(n_iter)

    print("Error: ", test(rms_tanlu.model))
    print('Test complete.')
    plt.show(block = True) # workaround to avoid freezing after closing plot
    return results

    
def run_cifar10(epochs = 20):
    print('\nStarting cifar10...\n')
    Xtrain, ytrain, Xtest, ytest = load_cifar10()
    
    cifar10_tanlu = Image((3, 32, 32)) >>\
    Conv((3, 32, 32), (64, 4, 4), activation=Tanlu) >> \
    Conv((64, 16, 16), (42, 4, 4), activation=Tanlu) >> \
    Conv((42, 8, 8), (64, 2, 2), activation=Tanlu) >> \
    Flatten() >> \
    Flatten() >> \
    Softmax(10)
    alphas_opt = [1, 1, 1]
    rms_tanlu = RMSProp(cifar10_tanlu, CrossEntropy())


    cifar10_opt = Image((3, 32, 32)) >>\
    Conv((3, 32, 32), (64, 4, 4), activation=Relu) >> \
    Conv((64, 16, 16), (42, 4, 4), activation=Relu) >> \
    Conv((42, 8, 8), (64, 2, 2), activation=Relu) >> \
    Flatten() >> \
    Flatten() >> \
    Softmax(10)
    rms_opt = RMSProp(cifar10_opt, CrossEntropy())

    return pipeline(Xtrain, ytrain, Xtest, ytest, rms_tanlu, rms_opt, epochs)

def run_cifar100(epochs = 20):
    print('\nStarting cifar100...\n')
    Xtrain, ytrain, Xtest, ytest = load_cifar100()
    
    cifar100_tanlu = Image((3, 32, 32)) >>\
    Conv((3, 32, 32), (64, 4, 4), activation=Tanlu) >> \
    Conv((64, 16, 16), (42, 4, 4), activation=Tanlu) >> \
    Conv((42, 8, 8), (64, 2, 2), activation=Tanlu) >> \
    Flatten() >> \
    Flatten() >> \
    Softmax(100)
    alphas_opt = [1, 1, 1]
    rms_tanlu = RMSProp(cifar10_tanlu, CrossEntropy())


    cifar100_opt = Image((3, 32, 32)) >>\
    Conv((3, 32, 32), (64, 4, 4), activation=Relu) >> \
    Conv((64, 16, 16), (42, 4, 4), activation=Relu) >> \
    Conv((42, 8, 8), (64, 2, 2), activation=Relu) >> \
    Flatten() >> \
    Softmax(100) # Flatten() >> \
    rms_opt = RMSProp(cifar10_opt, CrossEntropy())

    return pipeline(Xtrain, ytrain, Xtest, ytest, rms_tanlu, rms_opt, epochs)


def run_mnist(epochs = 20):
    print('\nStarting mnist...\n')
    Xtrain, ytrain, Xtest, ytest = load_mnist()
    
    mnist_tanlu = Image((1, 28, 28)) >> Conv((1, 28, 28), (10, 2, 2), activation=Tanlu) >> \
    Conv((10, 14, 14), (20, 2, 2), activation=Tanlu) >> Flatten() >> Softmax(10)

    rms_tanlu = RMSProp(mnist_tanlu, CrossEntropy())

    mnist_opt = Image((1, 28, 28)) >> Conv((1, 28, 28), (10, 2, 2), activation=Relu) >> \
    Conv((10, 14, 14), (20, 2, 2), activation=Tanh) >> Flatten() >> Softmax(10)

    rms_opt = RMSProp(mnist_opt, CrossEntropy())

    return pipeline(Xtrain, ytrain, Xtest, ytest, rms_tanlu, rms_opt, epochs)

#results_m = run_mnist(50)
results_10 = run_cifar10(70)
results_100 = run_cifar100(50)

/home/alex/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/nvcc_compiler.py:224: UserWarning: You have the cuda library directory in your lib_dirs. This has been known to cause problems and should not be done.
  warnings.warn("You have the cuda library directory in your "
Using gpu device 0: GeForce GTX 860M (CNMeM is enabled with initial size: 80.0% of memory, CuDNN 4004)


Starting cifar10...

 Iteration  Time  Loss Tanlu  Loss Optimal  Error Tanlu  Error Optimal  alpha_1  alpha_2  alpha_3
         0     2      2.3002        2.2907          0.9          0.894      0.5      0.5      0.5


/home/alex/anaconda2/lib/python2.7/site-packages/matplotlib/backend_bases.py:2437: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)



 Iteration   Time  Loss Tanlu  Loss Optimal  Error Tanlu  Error Optimal  alpha_1  alpha_2  alpha_3
      1000  123.4      1.1299         1.186        0.411          0.406     0.51     0.36     0.71
 Iteration   Time  Loss Tanlu  Loss Optimal  Error Tanlu  Error Optimal  alpha_1  alpha_2  alpha_3
      2000  245.9      0.6294        0.7055         0.35           0.35     0.58     0.38     0.71